# Performing a hyperparameter optimization for a text classifier

<a target="_blank" href="https://www.recogn.ai/biome-text/documentation/tutorials/Performing_a_hyperparameter_optimization_for_a_text_classifier.html"><img class="icon" src="https://www.recogn.ai/biome-text/assets/img/biome-isotype.svg" width=24 /></a>
[View on recogn.ai](https://www.recogn.ai/biome-text/documentation/tutorials/Performing_a_hyperparameter_optimization_for_a_text_classifier.html)

<a target="_blank" href="https://colab.research.google.com/github/recognai/biome-text/blob/master/docs/docs/documentation/tutorials/Performing_a_hyperparameter_optimization_for_a_text_classifier.ipynb"><img class="icon" src="https://www.tensorflow.org/images/colab_logo_32px.png" width=24 /></a>
[Run in Google Colab](https://colab.research.google.com/github/recognai/biome-text/blob/master/docs/docs/documentation/tutorials/Performing_a_hyperparameter_optimization_for_a_text_classifier.ipynb)

<a target="_blank" href="https://github.com/recognai/biome-text/blob/master/docs/docs/documentation/tutorials/Performing_a_hyperparameter_optimization_for_a_text_classifier.ipynb"><img class="icon" src="https://github.githubassets.com/images/modules/logos_page/GitHub-Mark.png" width=24 /></a>
[View source on GitHub](https://github.com/recognai/biome-text/blob/master/docs/docs/documentation/tutorials/Performing_a_hyperparameter_optimization_for_a_text_classifier.ipynb)

In this tutorial we will optimize the hyperparameters of the short-text classifier from [this tutorial](./Training_a_text_classifier.html).
We recommend to have a look at it first before going through the following tutorial.
For the Hyper-Parameter Optimization (HPO) we rely on the awesome [ray tune library](https://docs.ray.io/en/latest/tune.html#tune-index) that is **not** a dependency of *biome.text* and has to be installed additionally.

For a short introduction to HPO with Ray Tune you can have a look at this nice [talk](https://www.youtube.com/watch?v=VX7HvEoMrsA) by Richard Liaw. 
We will follow his terminology and use the name *trial* to refer to the training run of one set of hyperparameters. 

When running this tutorial in Google Colab, make sure to install *biome.text* and *ray tune* first:

In [ ]:
!pip install -U git+https://github.com/recognai/biome-text.git ray[tune]

Ignore warnings and don't forget to restart your runtime afterwards (*Runtime -> Restart runtime*).

::: tip Note

In this tutorial we will use a GPU by default.
So when running this tutorial in Google Colab, make sure that you request one (*Edit -> Notebook settings*).

:::

## Download the data and create the vocabulary

As a first step we will download the training and validation data to our local machine. 
This will save us some time in the long run, since we will perform the hyperparameter search on our local machine and frequently access the data.

In [ ]:
!curl -O https://biome-tutorials-data.s3-eu-west-1.amazonaws.com/text_classifier/business.cat.train.csv

In [ ]:
!curl -O https://biome-tutorials-data.s3-eu-west-1.amazonaws.com/text_classifier/business.cat.valid.csv

We will store the absolute path of the data to use them later on when creating our `DataSource`s.

In [ ]:
import os

In [ ]:
train_path = os.path.abspath("business.cat.train.csv")
valid_path = os.path.abspath("business.cat.valid.csv")

### Reuse the vocabulary

In order to be more efficient and speed things up, we will create the vocabulary beforehand and reuse it in every trial.
For this we have to create a `Pipeline` first, create the vocabulary from our `DataSource` and save it to a folder.

Let's start with defining the configuration of our pipeline (for details see [this tutorial](./Training_a_text_classifier.html)):

In [ ]:
from biome.text.data import DataSource

In [ ]:
labels = DataSource(train_path).to_dataframe().label.compute()

In [ ]:
pipeline_dict = {
    "name": "german_business_names",
    "tokenizer": {
        "text_cleaning": {
            "rules": ["strip_spaces"]
        }
    },
    "features": {
        "word": {
            "embedding_dim": 64,
            "lowercase_tokens": True,
        },
        "char": {
            "embedding_dim": 32,
            "lowercase_characters": True,
            "encoder": {
                "type": "gru",
                "num_layers": 1,
                "hidden_size": 32,
                "bidirectional": True,
            },
            "dropout": 0.1,
        },
    },
    "head": {
        "type": "TextClassification",
        "labels": list(labels.value_counts().index),
        "pooler": {
            "type": "gru",
            "num_layers": 1,
            "hidden_size": 32,
            "bidirectional": True,
        },
        "feedforward": {
            "num_layers": 1,
            "hidden_dims": [32],
            "activations": ["relu"],
            "dropout": [0.0],
        },
    },       
}

We will use this configuration dictionary to create our pipeline:

In [ ]:
from biome.text import Pipeline

In [ ]:
pl = Pipeline.from_config(pipeline_dict)

Next, we have to define the vocabulary configuration and create our vocabulary.


::: tip Note

If you want to optimize the vocabulary configuration in the hyperparameter search (for example, the `min_count` argument), you have to move the vocabulary creation to the `trainable` function below. That is, in each trial the vocabulary will be created anew.

:::

In [ ]:
from biome.text.configuration import VocabularyConfiguration, WordFeatures

In [ ]:
vocab_config = VocabularyConfiguration(sources=[DataSource(train_path)], min_count={WordFeatures.namespace: 20})

In [ ]:
pl.create_vocabulary(vocab_config)

To be able to reuse the vocabulary in each trial, we have to save it to a folder and store its absolute path:

In [ ]:
vocab_absolute_path = os.path.abspath("./vocabulary")

In [ ]:
pl.save_vocabulary(vocab_absolute_path)

## Implementing the callback for early stopping

In this tutorial we will use a trial scheduler that adaptively allocates resources to promising hyperparameter configurations by terminating less promising candidates early.
The early stopping mechanism requires the reporting of some metric during a trial.
For this we use an `EpochCallback` that is executed after each epoch and pass it on to the `Pipeline.train()` method.

Our `TuneReport` class simply reports some metrics back to tune, which in turn are used to define promising trials during the hyperparameter search:  

In [ ]:
from allennlp.training import EpochCallback
from ray import tune

In [ ]:
class TuneReport(EpochCallback):
    def __call__(self, trainer, metrics, epoch, is_master):
        # The EpochCallback is also called before the training starts (at epoch = -1)
        if epoch > -1:
            tune.report(validation_loss=metrics["validation_loss"], validation_accuracy=metrics["validation_accuracy"])

In [ ]:
tune_report = TuneReport()

## Defining the training loop

For the HPO with *biome.text* we will use the [funtion-based Trainable API](https://docs.ray.io/en/latest/tune/api_docs/trainable.html#tune-function-api) of Ray Tune.
Therefore, we have to define a `trainable` function that takes as input a configuration dictionary and executes a training run.

We will use the configuration dictionary to create a `Pipeline` and a `TrainerConfiguration` in order to optimize the learning rate.
In the `Pipeline.train()` method we will add our `tune_report` instance to the epoch callbacks, and completely silence the output of the training by setting `quiet=True`.
This avoids cluttering the output of the hyperparameter search and makes it easier to follow the progress.

::: tip Tip

Keep in mind that the learning rate "*is often the single most important hyper-parameter and one should always make sure that it has been tuned (up to approximately a factor of 2). ... If there is only time to optimize one hyper-parameter and one uses stochastic gradient descent, then this is the hyper-parameter that is worth tuning." ([Yoshua Bengio](https://arxiv.org/abs/1206.5533)).

:::

In [ ]:
from biome.text.configuration import TrainerConfiguration

In [ ]:
def trainable(config):
    pl = Pipeline.from_config(config["pipeline"], vocab_absolute_path)
    trainer_config = TrainerConfiguration(**config["trainer"])
    
    train_ds = DataSource(train_path)
    valid_ds = DataSource(valid_path)
    
    pl.train(
        output="output",
        training=train_ds,
        validation=valid_ds,
        trainer=trainer_config,
        epoch_callbacks=[tune_report],
        quiet=True,
    )

## Random search with a trial scheduler

In this tutorial we will perform a random search with the [Asynchronous Successive Halving Algorithm (ASHA)](https://blog.ml.cmu.edu/2018/12/12/massively-parallel-hyperparameter-optimization/) to schedule our trials.
The Ray Tune developers advocate this scheduler as a good starting point for its aggressive termination of low-performing trials.

To perform a random search (as well as a grid search) we simply have to replace the parameters we want to optimize with methods from the [Random Distributions API](https://docs.ray.io/en/latest/tune/api_docs/grid_random.html#random-distributions-api) and the [Grid Search API](https://docs.ray.io/en/latest/tune/api_docs/grid_random.html#grid-search-api), respectively.
For a complete description of both APIs and how they interplay with each other, see the corresponding section in the [Ray Tune docs](https://docs.ray.io/en/latest/tune/api_docs/grid_random.html).

In our case we will tune 4 parameters:
- the embedding dimension of our `word` feature
- the embedding dimension of our characters in our `char` feature
- the output dimension of the `char` feature (the hidden size of our GRU)
- and the learning rate

For the first 3 parameters we will provide discrete values from which Tune will sample randomly, while for the learning rate we will provide a continuous logarithmic range.
Since we want to directly compare the outcome of the optimization with the base configuration of the [underlying tutorial](./Training_a_text_classifier.html), we will fix the number of epochs to 3.

::: tip Note

By default we will use a GPU.
If you do not have one available, just comment out the line `"cuda_device": 0` in the trainer section of the dictionary below.

:::

In [ ]:
configs = {
    "pipeline": {
        "name": "german_business_names",
        "tokenizer": {
            "text_cleaning": {
                "rules": ["strip_spaces"]
            }
        },
        "features": {
            "word": {
                "embedding_dim": tune.choice([16, 32, 64]),
                "lowercase_tokens": True,
            },
            "char": {
                "embedding_dim": tune.choice([16, 32, 64]),
                "lowercase_characters": True,
                "encoder": {
                    "type": "gru",
                    "num_layers": 1,
                    "hidden_size": tune.choice([16, 32, 64]),
                    "bidirectional": True,
                },
                "dropout": 0.1,
            },
        },
        "head": {
            "type": "TextClassification",
            "labels": list(labels.value_counts().index),
            "pooler": {
                "type": "gru",
                "num_layers": 1,
                "hidden_size": 32,
                "bidirectional": True,
            },
            "feedforward": {
                "num_layers": 1,
                "hidden_dims": [32],
                "activations": ["relu"],
                "dropout": [0.0],
            },
        },       
    }, 
    "trainer": {
        "optimizer": {
            "type": "adam",
            "lr": tune.loguniform(0.001, 0.01),
        },
        "num_epochs": 3,
        "cuda_device": 0,
    },
}

As mentioned above, we will use the ASHA trial scheduler for our search.
To create an instance of the `ASHAScheduler` we have to specify the decisive metric for terminating low-performing trials and the mode of this metric (is the objective to *minimize* the metric, `min`, or to *maximize* it, `max`).
For a complete description of the configurations, see the [ASHAScheduler docs](https://docs.ray.io/en/latest/tune/api_docs/schedulers.html#asha-tune-schedulers-ashascheduler).

In [ ]:
from ray.tune.schedulers import ASHAScheduler

In [ ]:
asha = ASHAScheduler(metric="validation_loss", mode="min")

## Following the progress with tensorboard (optional)

Ray Tune automatically logs its results with [TensorBoard](https://www.tensorflow.org/tensorboard/).
We can take advantage of this and launch a TensorBoard instance before starting the hyperparameter search to follow its progress.

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir ./tune/trainable

## Start the hyperparameter search

Now we have everything ready to start our hyperparameter search with the `tune.run()` method.

The number of trials our search will go through depends on the `num_samples` parameter.
In our case, a random search, it equals the number of trials, whereas in the case of a grid search the total number of trials is `num_samples` times the grid configurations (see [the docs](https://docs.ray.io/en/latest/tune/api_docs/grid_random.html) for illustrative examples).

The number of parallel running trials depends on your `resources_per_trial` configuration and your local resources.
The default value is `{"cpu": 1, "gpu": 0}` and results in 8 parallel running trials on a machine with 8 CPUs.
You can also use fractional values, for example to share a GPU between 2 trials, pass on `{"gpu": 0.5}`. 

The `local_dir` parameter defines the output directory of the HPO results and will also contain the training results of each trial (that is the model weights and metrics).

::: tip Note

Keep in mind: to run your HPO on GPUs, you have to specify them in the `TrainerConfiguration` in the `trainable` function, as well as in the `resources_per_trial` parameter when calling `tune.run()`.
If you do not want to use a GPU, just set the value to 0 `{"cpu": 1, "gpu": 0}`.

:::

In [ ]:
analysis = tune.run(
    trainable, 
    config=configs, 
    scheduler=asha, 
    num_samples=1, 
    resources_per_trial={"cpu": 1, "gpu": 0.5},
    local_dir="./tune", 
)

## Checking the results (WIP)

Apart from the TensorBoard view, we can also use the `analysis` object returned by `tune.run()` to check the results of our HPO.

In [ ]:
analysis.get_best_config(metric="validation_loss", mode="min")

In [ ]:
analysis.dataframe(metric="validation_loss", mode="min")

- compare accuracy with basic tutorial
- real live example: increase epochs!